In [17]:
import re
import numpy as np
from urllib.request import urlopen
import json

In [18]:
input_day1 = set([])
with open('day1.txt') as f:
    for line in f:
        input_day1.add(int(line.strip()))
expected_set = {2020 - i for i in input_day1}
expected_val = tuple(input_day1.intersection(expected_set))
day1_answer = expected_val[0] * expected_val[1]
print(day1_answer)

for i in input_day1:
    for j in input_day1 - {i}:
        for k in input_day1 - {i, j}:
            if (i + j + k) == 2020:
                print(i, j, k, i*j*k)
                break

1016964
1216 295 509 182588480
1216 509 295 182588480
295 1216 509 182588480
295 509 1216 182588480
509 1216 295 182588480
509 295 1216 182588480


In [19]:
input_day2 = []
with open('day2.txt') as f:
    for line in f:
        split_contents = re.split(':*\s+', line.strip())
        input_day2.append(tuple(split_contents))

def _validateRule(expectedCount, char, passWd):
    expectedCount = re.split(r'-', expectedCount)
    minCount = int(expectedCount[0])
    maxCount = int(expectedCount[-1])
    return minCount <= passWd.count(char) <= maxCount


def obtain_validPasswords(passwordsSet):
    return [pwd_with_rule for pwd_with_rule in passwordsSet if _validateRule(*pwd_with_rule)]


validPasswords = obtain_validPasswords(input_day2)
print(len(validPasswords))


def _validateRule(possiblePositions, char, passWd):
    possiblePositions = re.split(r'-', possiblePositions)
    pos1 = int(possiblePositions[0]) - 1
    pos2 = int(possiblePositions[-1]) - 1
    if pos1 == pos2:
        print(f"Unique scenario:{possiblePositions}, {char}, {passWd}")
    return (passWd[pos1] == char) ^ (passWd[pos2] == char)

validPasswords2 = obtain_validPasswords(input_day2)
print(len(validPasswords2))

445
491


In [20]:
input_day3 = []
with open('day3.txt') as f:
    for line in f:
        input_day3.append(tuple(line.strip()))
        
def check_tree_crossings(right, down, txtmap):
    encountered_trees = [row for i, row in enumerate(txtmap[::down])
                        if row[(i * right) % len(row)] == '#'
                    ]
    return encountered_trees

right_down_slopes = [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)]

various_tree_crossings = [check_tree_crossings(r, d, input_day3) for r, d in right_down_slopes]

print(len(various_tree_crossings[1]))

num_tree_crossings = [len(x) for x in various_tree_crossings]

num_tree_crossings
print(np.prod(num_tree_crossings, dtype=np.int64))

214
8336352024


In [21]:
input_day4 = [dict()]
def _convert_string2dict(string):
    return {matches[0]: matches[1] for matches in re.findall(r"(\S+)\s*:\s*(\S+)", string)}


with open('day4.txt') as f:
    for line in f:
        line = line.strip()
        if re.match(r'^\s*$', line):
            input_day4.append(dict())
        else:
            input_day4[-1].update(_convert_string2dict(line))

mandatory_fields = {
    'byr' : '(Birth Year)',
    'iyr' : '(Issue Year)',
    'eyr' : '(Expiration Year)',
    'hgt' : '(Height)',
    'hcl' : '(Hair Color)',
    'ecl' : '(Eye Color)',
    'pid' : '(Passport ID)',
    'cid' : '(Country ID)',
}

reqd_fields = set(mandatory_fields.keys()) - {'cid'}
valid_passports = [passPt for passPt in input_day4 
                   if reqd_fields.issubset(set(passPt.keys()))
                  ]
print(len(valid_passports))


def _check_year(year, min_yr, max_yr):
    try:
        return min_yr <= int(year) <= max_yr
    except:
        return False

def _check_height(height):
    m = re.search(r"([0-9]+)(cm|in)", height, flags=re.IGNORECASE)
    if m:
        if m.group(2) == 'in':
            return 59 <= int(m.group(1)) <= 76
        else:
            return 150 <= int(m.group(1)) <= 193
    else:
        return False
    
def _check_haircolor(hairClr):
    return bool(re.match(r'#[0-9a-f]{6}', hairClr))

def _check_eyecolor(eyeClr, allClrs=('amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth')):
    return eyeClr in allClrs

def _check_passportID(passPtId):
    try:
        int(passPtId) > 0
    except:
        return False
    # return len(passPtId.zfill(9)) == 9 ## They don't mean patch zeros
    return len(passPtId) == 9
    
valid_passports2 = [passPt for passPt in valid_passports if 
                    (_check_year(passPt['byr'], 1920, 2002) and
                    _check_year(passPt['iyr'], 2010, 2020) and
                    _check_year(passPt['eyr'], 2020, 2030) and
                    _check_height(passPt['hgt']) and
                    _check_haircolor(passPt['hcl']) and
                    _check_eyecolor(passPt['ecl']) and
                    _check_passportID(passPt['pid']))
                   ]

print(len(valid_passports2))

250
158


In [22]:
def get_row_decimal_repr(chars, upchar='B', dnchar='F'):
    return int(re.sub(upchar, '1', re.sub(dnchar, '0', chars)), 2)

def get_col_decimal_repr(chars):
    return get_row_decimal_repr(chars, upchar='R', dnchar='L')

def get_unique_seat_id(chars):
    return get_row_decimal_repr(chars[:7]) * 8 + get_col_decimal_repr(chars[7:])


input_day5 = []
with open('day5.txt') as f:
    for line in f:
        input_day5.append(line.strip())

unique_seat_ids = [get_unique_seat_id(code) for code in input_day5]
print(max(unique_seat_ids))


all_possible_unique_ids = [r * 8 + l for r in range(2**7) for l in range(2**3)]
missing_unique_ids = set(all_possible_unique_ids) - set(unique_seat_ids)
for my_id in missing_unique_ids:
    if (my_id + 1 in unique_seat_ids) and (my_id - 1 in unique_seat_ids):
        print(f"My probable id: {my_id}")

928
My probable id: 610


In [23]:
input_day6 = [set()]

with open('day6.txt') as f:
    for line in f:
        line = line.strip()
        if re.match(r'^\s*$', line):
            input_day6.append(set())
        else:
            input_day6[-1].update(set(line))
            
counts_of_declarations = [len(chars) for chars in input_day6]
print(sum(counts_of_declarations))

input_day6_new = [set()]
grpMembers = 0
with open('day6.txt') as f:
    for line in f:
        line = line.strip()
        if re.match(r'^\s*$', line):
            input_day6_new.append(set())
            grpMembers = 0
        else:
            grpMembers += 1
            if grpMembers == 1:
                input_day6_new[-1].update(set(line))
            else:
                input_day6_new[-1] = input_day6_new[-1].intersection(set(line))
                            
counts_of_declarations_new = [len(chars) for chars in input_day6_new]
print(sum(counts_of_declarations_new))

6382
3197


In [24]:
from collections import defaultdict

def process_bags(string):
    bagcolorRe = r"(\S+)\s+(\S+)\s+bag[s]*"
    bagnumcolorRe = r"((\d+)\s+)*(\S+)\s+(\S+)\s+bag[s]*"
bagNumColorRe = r"((\d+)\s+)*(\S+\s+\S+)\s+bag[s]*"    


def convert_to_contents_dict(fileContents, bagNumColorRe=r"((\d+)\s+)*(\S+\s+\S+)\s+bag[s]*"):
    converted_dict = defaultdict(dict)
    for line in fileContents:
        line = line.strip()
        lineContents = re.split(r"contain[s]*", line)
        m = re.match(bagNumColorRe, lineContents[0])
        primKey = m.group(3)
        for contents in re.split(r"\s*,\s*", lineContents[-1]):
            m = re.match(bagNumColorRe, contents.strip())
            #print(contents)
            if m.group(2):
                converted_dict[primKey][m.group(3)] = int(m.group(2))
            else:
                converted_dict[primKey] = {}
    return converted_dict

#input_day7 = defaultdict(dict)
with open('day7.txt') as f:
    input_day7 = convert_to_contents_dict(f)  
# print(input_day7.keys())
# print(json.dumps(input_day7, indent=4))
# shown_example = [
#                  'light red bags contain 1 bright white bag, 2 muted yellow bags.',
#                  'dark orange bags contain 3 bright white bags, 4 muted yellow bags.',
#                  'bright white bags contain 1 shiny gold bag.',
#                  'muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.',
#                  'shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.',
#                  'dark olive bags contain 3 faded blue bags, 4 dotted black bags.',
#                  'vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.',
#                  'faded blue bags contain no other bags.',
#                  'dotted black bags contain no other bags.'
#                 ]
# shown_example_dict = convert_to_contents_dict(shown_example)
# print(json.dumps(shown_example_dict, indent=4))


def find_parent_colors(bagsDict, color):
    resultSet = set()
    for key, values in bagsDict.items():
        if color in values:
            resultSet.add(key)
    grandparSet = set()
    for primParent in resultSet:
        grandparSet.update(find_parent_colors(bagsDict, primParent))
    resultSet.update(grandparSet)
    return resultSet

# test = find_parent_colors(shown_example_dict, 'shiny gold')
# print(test)

possible_bags_with_shinyGold = find_parent_colors(input_day7, 'shiny gold')
print(len(possible_bags_with_shinyGold))

def obtain_total_bag_contents(bagsDict, color):
    total = 0
    for subColor, count in bagsDict[color].items():
        total += count + count * obtain_total_bag_contents(bagsDict, subColor)
    return total

# shown_example = ['shiny gold bags contain 2 dark red bags.',
#                  'dark red bags contain 2 dark orange bags.',
#                  'dark orange bags contain 2 dark yellow bags.',
#                  'dark yellow bags contain 2 dark green bags.',
#                  'dark green bags contain 2 dark blue bags.',
#                  'dark blue bags contain 2 dark violet bags.',
#                  'dark violet bags contain no other bags.']
# shown_example_dict = convert_to_contents_dict(shown_example)
# obtain_total_bag_contents(shown_example_dict, 'shiny gold')
print(obtain_total_bag_contents(input_day7, 'shiny gold'))

211
12414
